## Import Libraries

In [1]:
# standard setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import time

# useful helper function
from helpers import imshow

## Real time spine tracking and augmentation

In [57]:


query_path = 'book2.jpg'
query = cv2.imread(query_path, 0)



target_height = query.shape[0]



scaling_factor_q = 1.0*target_height/query.shape[0]
query = cv2.resize(query, None, fx = scaling_factor_q, fy = scaling_factor_q, interpolation = cv2.INTER_AREA)
print query.shape


AR_image_path = 'cover4.jpg'
AR_image = cv2.imread(AR_image_path)
scaling_factor_ar = 1.0*target_height/AR_image.shape[0]
AR_image = cv2.resize(AR_image, None, fx = scaling_factor_ar, fy = scaling_factor_ar, interpolation = cv2.INTER_AREA)
print AR_image.shape


# SIFT keypoint detector
sift = cv2.xfeatures2d.SIFT_create()

# Import the query image and compute SIFT keypoints and descriptors
kp_query, des_query = sift.detectAndCompute(query, None)  

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)

# Camera settings
camera = cv2.VideoCapture(0)
# reduce frame size to speed it up
w = 640
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
camera.set(cv2.CAP_PROP_EXPOSURE,-4) 

while True:
    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # keypoints and descriptors for video frame 
    
    kp_scene, des_scene =  sift.detectAndCompute(gray,None)
       
    # Match scene descriptors with query descriptors
    matches = flann.knnMatch(des_query,des_scene,k=2)

    # ratio test as per Lowe's paper
    good_matches = []
    # Each member of the matches list must be checked whether two neighbours really exist.
    for m_n in matches:
        if len(m_n) != 2:
            continue
        (m,n) = m_n
        if m.distance < 0.7*n.distance:
            good_matches.append(m)
    
#     print "{} matches found".format(len(good_matches))

    # If enough matches found ...
    if len(good_matches) > 15:
        
        # Source points and destnation points
        src_pts = np.float32([kp_query[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_scene[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        
        # Compute Homography
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        rows,cols = frame.shape[:2]
        dst = cv2.warpPerspective(AR_image, M, (cols, rows));

        frame[np.nonzero(dst)] = dst[np.nonzero(dst)]
    
        
#         # Draw a rectangle that marks the found model in the frame
#         h, w = query.shape
#         pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
#         # project corners into frame
#         dst = cv2.perspectiveTransform(pts, M)
    
#         # Draw lines
#         cv2.polylines(frame, [np.int32(dst)], True, (0, 255, 255), 3, cv2.LINE_AA) 
        
    
    if cv2.waitKey(5) == 32:
        imshow(frame)        
    elif cv2.waitKey(5) == 27:
        break  
   
    #frame = cv2.flip(frame, 1)
    cv2.imshow("SIFT Frame", frame)
    
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window



(668, 88)
(668, 503, 3)


-1

In [49]:

camera = cv2.VideoCapture(1)
print camera.isOpened()
camera.release()


True
